# Setup

In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)

# Load all gated subsets

## CD8 subsets

In [ ]:
cd8_na <- readRDS('../Seurat_Objects/cd8_naive.rds')
cd8_cm <- readRDS('../Seurat_Objects/cd8_cm.rds')
cd8_em <- readRDS('../Seurat_Objects/cd8_em.rds')
cd8_emra <- readRDS('../Seurat_Objects/cd8_emra.rds')
cd8_unk <- readRDS('../Seurat_Objects/cd8_unk.rds')

## CD4 Subsets

In [ ]:
cd4_na <- readRDS('../Seurat_Objects/cd4_naive.rds')
cd4_cm <- readRDS('../Seurat_Objects/cd4_cm.rds')
cd4_em <- readRDS('../Seurat_Objects/cd4_em.rds')
cd4_emra <- readRDS('../Seurat_Objects/cd4_emra.rds')
cd4_unk <- readRDS('../Seurat_Objects/cd4_unk.rds')
treg <- readRDS('../Seurat_Objects/treg.rds')

## Merge
### Check Counts

In [ ]:
sp_merge <- merge(cd8_na, c(cd8_cm, cd8_em, cd8_emra, cd8_unk, cd4_na, cd4_cm, cd4_em, cd4_emra, treg, cd4_unk))

In [ ]:
sp_merge

## Remove subsets to save memory

In [ ]:
rm(cd8_na)
rm(cd8_cm)
rm(cd8_em)
rm(cd8_emra)
rm(cd8_unk)
rm(cd4_na)
rm(cd4_em)
rm(cd4_cm)
rm(cd4_emra)
rm(cd4_unk)
rm(treg)

# Add Metadata

In [ ]:
sp_merge <- SetIdent(sp_merge, value = 'pbmc_sample_id')
sp_merge <- RenameIdents(sp_merge,
                           'PB00173-02' = 'Positive', #validated
                           'PB00192-02' = 'Negative', #validated
                           'PB00197-02' = 'Negative', #validated
                           'PB00593-04' = 'Positive', #validated
                           'PB00172-02' = 'Positive',
                           'PB00182-02' = 'Negative',
                           'PB00323-02' = 'Positive',
                           'PB00807-02' = 'Positive',
                           'PB00124-02' = 'Positive',
                           'PB00127-02' = 'Negative',
                           'PB00334-03' = 'Negative',
                           'PB00353-03' = 'Negative',
                           'PB00368-04' = 'Positive',
                           'PB00377-03' = 'Negative',
                           'PB00545-02' = 'Negative',
                           'PB00599-02' = 'Positive')
sp_merge$CMV <- Idents(sp_merge)

In [ ]:
sp_merge <- SetIdent(sp_merge, value = 'pbmc_sample_id')
sp_merge <- RenameIdents(sp_merge,
                           'PB00173-02' = 'UP1006', #validated
                           'PB00192-02' = 'UP1007', #validated
                           'PB00197-02' = 'UP1010', #validated
                           'PB00593-04' = 'BR2002', #validated
                           'PB00172-02' = 'UP1002',
                           'PB00182-02' = 'UP1003',
                           'PB00323-02' = 'BR2004',
                           'PB00807-02' = 'UP1005',
                           'PB00124-02' = 'UP1001',
                           'PB00127-02' = 'UP1004',
                           'PB00334-03' = 'BR2009',
                           'PB00353-03' = 'BR2008',
                           'PB00368-04' = 'BR2005',
                           'PB00377-03' = 'BR2015',
                           'PB00545-02' = 'BR2042',
                           'PB00599-02' = 'BR2052')
sp_merge$subject_id <- Idents(sp_merge)

In [ ]:
sp_merge$age_cmv <- paste0(sp_merge$pediatric_senior,"_CMV-",sp_merge$CMV)

In [ ]:
saveRDS(sp_merge, file = '../Seurat_Objects/sp_merge_only.rds')

# Gated Cell Type Proportions (BoxPlot)

In [ ]:
sp_merge <- readRDS(file = '../Seurat_Objects/sp_merge_only.rds')

In [ ]:
metadata <- sp_merge@meta.data

In [ ]:
gated_counts <- dplyr::count(metadata, gating_celltype, pbmc_sample_id, pediatric_senior)

In [ ]:
head(gated_counts)

In [ ]:
sum_counts <- metadata %>%
                dplyr::count(pediatric_senior, pbmc_sample_id)
head(sum_counts)

In [ ]:
joined_counts <- left_join(gated_counts, sum_counts, by = 'pbmc_sample_id')
head(joined_counts)

In [ ]:
joined_counts$perc <- joined_counts$n.x/joined_counts$n.y
colnames(joined_counts) <- c('gating_celltype','pbmc_sample_id','pediatric_senior','count','pediatric_senior_repeat','sample_total_count','perc')
head(joined_counts)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
ggplot(joined_counts, aes(x = reorder(gating_celltype,-perc), y = perc, fill = pediatric_senior)) + 
    geom_boxplot() + 
    scale_y_log10() + 
    geom_dotplot(binaxis='y', stackdir='center',
                 position=position_dodge(), binwidth = 1/40) + 
    theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5, size = 15),
          axis.text.y = element_text(size = 15)) +
    guides(fill = guide_legend(title = 'Pediatric / Senior')) +
    xlab("Gating Cell Types") + 
    ylab("Percent")

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
ggplot(joined_counts, aes(x = reorder(gating_celltype,-perc), y = perc, fill = pediatric_senior)) + 
    geom_boxplot() + 
    #scale_y_log10() + 
    geom_dotplot(binaxis='y', stackdir='center',
                 position=position_dodge(), binwidth = 1/200) + 
    theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5, size = 15),
          axis.text.y = element_text(size = 15)) +
    guides(fill = guide_legend(title = 'Pediatric / Senior')) +
    xlab("Gating Cell Types") + 
    ylab("Percent")

In [ ]:
write.csv(joined_counts, file = 'gating_counts.csv')

In [ ]:
sessionInfo()